<a href="https://colab.research.google.com/github/annisafitribas/ft_credit_home/blob/main/ft_credit_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Persiapan**

In [33]:
import os, sys, time, json
import warnings
warnings.filterwarnings('ignore')

WORKDIR = '/content/home_credit_task'
os.makedirs(WORKDIR, exist_ok=True)
print("WORKDIR:", WORKDIR)

WORKDIR: /content/home_credit_task


setup awal environment di Python

## Pengecekan

In [34]:
# Colab-friendly installs (only if missing)
try:
    import gdown
except Exception:
    !pip install -q gdown
    import gdown

# ML libs: install if missing
try:
    import lightgbm as lgb
except Exception:
    !pip install -q lightgbm
    import lightgbm as lgb

try:
    import xgboost as xgb
except Exception:
    !pip install -q xgboost
    import xgboost as xgb

try:
    from catboost import CatBoostClassifier
except Exception:
    !pip install -q catboost
    from catboost import CatBoostClassifier

try:
    from pptx import Presentation
    from pptx.util import Inches, Pt
except Exception:
    !pip install -q python-pptx
    from pptx import Presentation
    from pptx.util import Inches, Pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00


blok instalasi otomatis untuk beberapa library penting, khususnya agar bisa langsung jalan di Google Colab atau environment lain yang mungkin belum punya library tersebut.

Berikut detail tiap bagian:

## Library untuk plotting dan data, Library dari scikit-learn (sklearn)

In [35]:
# plotting / data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import joblib

# sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, classification_report, confusion_matrix, roc_curve, auc,
                             precision_recall_curve, average_precision_score)
from sklearn.impute import SimpleImputer

EDA & plotting → matplotlib, pandas, numpy
Preprocessing & scaling → StandardScaler, SimpleImputer
Modeling → Logistic Regression, Random Forest
Evaluation → metrics scikit-learn
Utilities → joblib, pathlib

## Pembantu (helper function)

In [36]:
def savefig(fig, filename):
    path = os.path.join(WORKDIR, filename)
    fig.savefig(path, bbox_inches='tight')
    print('Saved plot:', path)

Fungsi ini memudahkan menyimpan semua plot ke folder kerja secara konsisten tanpa harus menulis path lengkap setiap kali.